# Assignment Week 1 - Group 5

### Noelani Roy, Yihong Qiu, Cosimo Cambi, Craig Perkins

In [1]:
import pandas as pd
import numpy as np

In [2]:
# main libraries
import pandas as pd
import numpy as np
from datetime import date
import geopy.distance
from math import sin, cos, sqrt, atan2, radians, log
import imblearn
from numpy import mean, where
from collections import Counter
import qgrid

# visual libraries
import seaborn as sns 
import matplotlib.pyplot as plt

# sklearn libraries
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [8]:
filename = "../fraudTrain.csv"
filename2 = "../fraudTest.csv"

df = pd.read_csv(filename)
df2 = pd.read_csv(filename2)

fraud_df = pd.concat([df, df2])

# print(fraud_df)

In [4]:
fraud_df.shape

(1852394, 23)

In [9]:
from datetime import date
import geopy.distance

from math import sin, cos, sqrt, atan2, radians, log

def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

def calculate_distance(row):
    coords_1 = (row['lat'], row['long'])
    coords_2 = (row['merch_lat'], row['merch_long'])
    return geopy.distance.geodesic(coords_1, coords_2).km

# Answer from https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
# The answers above are based on the Haversine formula, which assumes the earth is a sphere,
# which results in errors of up to about 0.5% (according to help(geopy.distance)). 
# Vincenty distance uses more accurate ellipsoidal models such as WGS-84, and is implemented in geopy. For example,
def calculate_distance2(row):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(row['lat'])
    lon1 = radians(row['long'])
    lat2 = radians(row['merch_lat'])
    lon2 = radians(row['merch_long'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    print(distance)


# First derive columns
if 'trans_date_trans_time' in fraud_df.columns:
    fraud_df['txn_datetime'] =  pd.to_datetime(fraud_df['trans_date_trans_time'], format='%Y-%m-%d %H:%M:%S')

if 'dob' in fraud_df.columns:
    fraud_df['age'] = [calculate_age(d) for d in pd.to_datetime(fraud_df['dob'], format='%Y-%m-%d')]
    
if set(['lat', 'long', 'merch_lat', 'merch_long']).issubset(set(fraud_df.columns)):
     fraud_df['distance'] = [calculate_distance(row) for _, row in fraud_df.iterrows()]
        
fraud_df['hour'] = fraud_df['txn_datetime'].dt.hour
fraud_df['day'] = fraud_df['txn_datetime'].dt.day
fraud_df['month'] = fraud_df['txn_datetime'].dt.month
fraud_df['year'] = fraud_df['txn_datetime'].dt.year

fraud_df['log_amt'] = [log(n) for n in fraud_df['amt']]
    

# Drop the columns used to derive new features
fraud_df.drop([
    'Unnamed: 0',
    'trans_num',
    'unix_time',
    'first',
    'last',
    'street',
    'city',
    'state',
    'zip',
    'dob',
    'trans_date_trans_time',
    'lat',
    'long',
    'merch_lat',
    'merch_long'
], axis=1, errors='ignore', inplace=True)


cols = list(fraud_df.columns)
cols.insert(0, cols.pop(cols.index("txn_datetime")))
cols.append(cols.pop(cols.index('is_fraud')))
fraud_df = fraud_df[cols]

In [11]:
fraud_df.to_csv('fraudFeatures.csv', index=False)

In [3]:
filename1 = "../week2/fraudFeatures.csv"
fraud_df = pd.read_csv(filename1)

In [4]:
cat_col = ['category', 'gender']
num_col = [
    'age',
    'distance',
    'year',
    'month',
    'day',
    'hour',
    'city_pop'
]

In [5]:
# This one hot encodes the categorical columns and create a new variable to hold the nex column headers

cat_col_onehotencode = []

if len(cat_col) > 0: 
    cat_onehotencode = pd.get_dummies(fraud_df[cat_col], drop_first=True)
    cat_col_onehotencode = list(cat_onehotencode.columns)
    fraud_df = pd.concat([fraud_df,cat_onehotencode], axis = 1)

In [6]:
fraud_df.drop([
    'txn_datetime',
    'cc_num',
    'job',
    'log_amt',
    'merchant',
    'category',
    'gender',
], axis=1, errors='ignore', inplace=True)

In [ ]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# fraud_df['cc_num'] = le.fit_transform(fraud_df['cc_num']) 
# fraud_df['merchant'] = le.fit_transform(fraud_df['merchant']) 
# fraud_df['category'] = le.fit_transform(fraud_df['category']) 
# fraud_df['job'] = le.fit_transform(fraud_df['job']) 
# fraud_df['gender'] = le.fit_transform(fraud_df['gender']) 

In [7]:
import qgrid

qgrid.show_grid(fraud_df.head(100), grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [9]:
fraud_df["is_fraud"].value_counts()

0    1842743
1       9651
Name: is_fraud, dtype: int64

In [19]:
from imblearn.under_sampling import NearMiss

nm = NearMiss()

feature_cols = [col for col in fraud_df.columns if col not in ["is_fraud"]]
X = fraud_df[feature_cols] # Features
y = fraud_df["is_fraud"] # Target variable

X_res, y_res = nm.fit_sample(X, y)

In [20]:
from collections import Counter
from sklearn.datasets import make_classification
X2, y2 = make_classification(n_samples=5000, n_features=2, n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=3,
                           n_clusters_per_class=1,
                           weights=[0.01, 0.05, 0.94],
                          class_sep=0.8, random_state=0)
print(sorted(Counter(y).items()))
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X2, y2)
print(sorted(Counter(y_resampled).items()))
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X2, y2)
print(sorted(Counter(y_resampled).items()))

[(0, 1842743), (1, 9651)]
[(0, 4060), (1, 4381), (2, 3502)]
[(0, 4499), (1, 4566), (2, 4413)]


In [8]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# os = RandomOverSampler(sampling_strategy = 0.01) # Ratio of minority class / majority class
us = RandomUnderSampler(sampling_strategy = 0.25) # Ratio of minority class / majority class

feature_cols = [col for col in fraud_df.columns if col not in ["is_fraud"]]
X = fraud_df[feature_cols] # Features
y = fraud_df["is_fraud"] # Target variable

pipeline = Pipeline([
    # ('os', os),
    ('us', us)
])

X_resampled, y_resampled = pipeline.fit_resample(X, y)

In [25]:
# print(X_res.shape)
# print(y_res.shape)

fraud_df["is_fraud"].value_counts()

0    1842743
1       9651
Name: is_fraud, dtype: int64

In [8]:
y_resampled.value_counts()

0    137871
1      9651
Name: is_fraud, dtype: int64

In [9]:
import qgrid

qgrid.show_grid(X_resampled.head(100), grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=1)

In [11]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

In [13]:
split = int(len(y_test)/2)

inputX = X_train.to_numpy()
inputY = y_train.to_numpy()
inputX_valid = X_test.to_numpy()[:split]
inputY_valid = y_test.to_numpy()[:split]
inputX_test = X_test.to_numpy()[split:]
inputY_test = y_test.to_numpy()[split:]

In [17]:
# Number of input nodes.
input_nodes = 37

# Multiplier maintains a fixed ratio of nodes between each layer.
mulitplier = 1.5 

# Number of nodes in each hidden layer
hidden_nodes1 = 18
hidden_nodes2 = round(hidden_nodes1 * mulitplier)
hidden_nodes3 = round(hidden_nodes2 * mulitplier)

# Percent of nodes to keep during dropout.
tf.compat.v1.disable_eager_execution()
pkeep = tf.compat.v1.placeholder(tf.float32)

In [22]:
x = tf.compat.v1.placeholder(tf.float32, [None, input_nodes])

# layer 1
W1 = tf.Variable(tf.compat.v1.truncated_normal([input_nodes, hidden_nodes1], stddev = 0.15))
b1 = tf.Variable(tf.zeros([hidden_nodes1]))
y1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)

# layer 2
W2 = tf.Variable(tf.compat.v1.truncated_normal([hidden_nodes1, hidden_nodes2], stddev = 0.15))
b2 = tf.Variable(tf.zeros([hidden_nodes2]))
y2 = tf.nn.sigmoid(tf.matmul(y1, W2) + b2)

# layer 3
W3 = tf.Variable(tf.compat.v1.truncated_normal([hidden_nodes2, hidden_nodes3], stddev = 0.15)) 
b3 = tf.Variable(tf.zeros([hidden_nodes3]))
y3 = tf.nn.sigmoid(tf.matmul(y2, W3) + b3)
y3 = tf.nn.dropout(y3, pkeep)

# layer 4
W4 = tf.Variable(tf.compat.v1.truncated_normal([hidden_nodes3, 2], stddev = 0.15)) 
b4 = tf.Variable(tf.zeros([2]))
y4 = tf.nn.softmax(tf.matmul(y3, W4) + b4)

# output
y = y4
y_ = tf.compat.v1.placeholder(tf.float32, [None, 2])

In [23]:
training_epochs = 5 # should be 2000, it will timeout when uploading
training_dropout = 0.9
display_step = 1 # 10 
n_samples = y_train.shape[0]
batch_size = 2048
learning_rate = 0.005

In [29]:
# Cost function: Cross Entropy
cost = - tf.reduce_sum(y_ * tf.compat.v1.log(y))

# We will optimize our model via AdamOptimizer
optimizer = tf.optimizers.Adam(learning_rate).minimize(cost, var_list=[W1, b1])

# Correct prediction if the most likely value (Fraud or Normal) from softmax equals the target value.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

TypeError: 'Tensor' object is not callable

In [11]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(X_train.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               5888      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 137,729
Trainable params: 137,729
Non-trainable params: 0
__________________________________________________

In [13]:
counts = y_train.value_counts()

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

print(weight_for_0)
print(weight_for_1)

3.697814591576378e-05
0.00014847809948032666


In [14]:
import numpy as np
np.set_printoptions(suppress=True)

metrics = [
    keras.metrics.FalseNegatives(),
    keras.metrics.FalsePositives(),
    keras.metrics.TrueNegatives(),
    keras.metrics.TruePositives(),
    keras.metrics.Precision(),
    keras.metrics.Recall(),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    X_train,
    y_train,
    batch_size=2048,
    epochs=10,
    verbose=1,
    callbacks=callbacks,
    validation_data=(X_test, y_test),
    # class_weight=class_weight,
)

Epoch 1/10
17/17 [==============================] - 1s 54ms/step - loss: 2218.4282 - false_negatives_1: 4819.0000 - false_positives_1: 6318.0000 - true_negatives_1: 20725.0000 - true_positives_1: 1916.0000 - precision_1: 0.2327 - recall_1: 0.2845 - val_loss: 10.1849 - val_false_negatives_1: 1992.0000 - val_false_positives_1: 71.0000 - val_true_negatives_1: 11490.0000 - val_true_positives_1: 924.0000 - val_precision_1: 0.9286 - val_recall_1: 0.3169
Epoch 2/10
17/17 [==============================] - 0s 25ms/step - loss: 22.4393 - false_negatives_1: 3779.0000 - false_positives_1: 4424.0000 - true_negatives_1: 22619.0000 - true_positives_1: 2956.0000 - precision_1: 0.4005 - recall_1: 0.4389 - val_loss: 3.9500 - val_false_negatives_1: 1751.0000 - val_false_positives_1: 139.0000 - val_true_negatives_1: 11422.0000 - val_true_positives_1: 1165.0000 - val_precision_1: 0.8934 - val_recall_1: 0.3995
Epoch 3/10
17/17 [==============================] - 0s 24ms/step - loss: 4.8220 - false_negatives